In [ ]:
import numpy as np
import pandas as pd
import csv
import h5py
import os
import requests
import tempfile
import time
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
df_ratings = pd.read_csv('MovieLens_ratings.csv')

In [ ]:
def rating_splitter(df):
    
    df['liked'] = np.where(df['rating']>=4, 1, 0)
    df['movieId'] = df['movieId'].astype('str')
    gp_user_like = df.groupby(['liked', 'userId'])

    return ([gp_user_like.get_group(gp)['movieId'].tolist() for gp in gp_user_like.groups])

In [ ]:
pd.options.mode.chained_assignment = None
splitted_movies = rating_splitter(df_ratings)

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim
assert gensim.models.word2vec.FAST_VERSION > -1

In [ ]:
import random

for movie_list in splitted_movies:
    random.shuffle(movie_list)

In [ ]:
from gensim.models import Word2Vec
import datetime
start = datetime.datetime.now()

model = Word2Vec(sentences = splitted_movies, 
                 epochs = 5,
                 min_count = 1, 
                 vector_size = 200,
                 workers = 32,
                 sg = 1, 
                 hs = 0, 
                 negative = 5, 
                 window = 9999999,
                 seed = 0)

print("Time passed: " + str(datetime.datetime.now()-start))

In [ ]:
model.save('./model/item2vec_MovieLens_wo_split')